In [1]:
%matplotlib inline

# Welcome to exercise 06
1) 한국어 뉴스 데이터를 모아봅시다<br>
2) 한국어 뉴스 데이터셋을 `torchtext.datasets.text_classification`처럼 작동하도록 전처리해봅시다<br>
3) 모델을 무사히 돌려봅시다

## 1) 한국어 뉴스 데이터를 모아봅시다
Daum에서 섹션별로 뉴스를 긁어와 봅시다

### crawling 

In [2]:
from bs4 import BeautifulSoup
from downloads import *
import requests
import time
import urllib
import os

In [3]:
url = download("https://media.daum.net")
dom = BeautifulSoup(url.text,"lxml")
lists = [_["href"] for _ in dom.select(".link_gnb")]

In [4]:
for section in lists[1:5]:
    try:
        os.mkdir("./scraping/")
    except:
        None
    try:
        os.mkdir("./scraping/"+section)
        print(str(section)+" folder is created.")
    except:
        None
    print(section)
    URL = download("https://media.daum.net"+section)
    DOM = BeautifulSoup(URL.text,"lxml")
    artcl_list = [_ for _ in DOM.select(".tit_thumb a,.item_relate a") if _.has_attr("href")]
    for _ in artcl_list:
        DOM = BeautifulSoup(download(_["href"]).text,"lxml")
        try:
            title = DOM.select_one("h3.tit_view").text
            summary = " ".join([_.text for _ in DOM.select("strong.summary_view") if _.has_attr("text")])
            contents = " ".join([_.text for _ in DOM.select("div#harmonyContainer p")])
            link = _["href"]
            code = link.split("v/")[-1]
            with open("./scraping/"+section+'/'+str(URL.url.split("net/")[-1])+code+".txt","w",encoding="UTF8") as f:
                f.write(title)
                f.write(summary)
                f.write(contents)
                f.close()
        except:
            continue

/society
/politics
/economic
/foreign


## Data Loading
저장된 txt 파일을 섹션을 key로 갖는 딕셔너리에 넣읍시다 

In [2]:
def read_txt(file):
    return open(file).readline()

In [3]:
from glob import glob
from collections import defaultdict

In [4]:
subject = defaultdict(list)
for folder in glob('scraping/*'):
    for file in glob(folder+'/*'):
        subject[folder.split('/')[1]].append(read_txt(file).split())

In [5]:
subject_number = dict(enumerate(list(subject.keys())))

In [6]:
subject_number_inverse = {value:key for key, value in subject_number.items()}

In [7]:
subject = {subject_number_inverse[key]:value for key, value in subject.items()}

In [8]:
subject.keys()

dict_keys([0, 1, 2, 3])

### ngrams를 이용하여 데이터 불러오기

Bag of ngrams 피쳐는 지역(local) 단어 순서에 대한 부분적인 정보를 포착하기 위해 적용합니다.
실제 상황에서는 bi-gram이나 tri-gram은 단 하나의 단어를 이용하는 것보다 더 많은 이익을 주기 때문에 적용됩니다.
예를 들면 다음과 같습니다.

   "load data with ngrams"
   Bi-grams 결과: "load data", "data with", "with ngrams"
   Tri-grams 결과: "load data with", "data with ngrams"

``TextClassification`` 데이터셋은 ngrams method을 지원합니다. ngrams을 2로 설정하면,
데이터셋 안의 예제 텍스트는 각각의(single) 단어들에 bi-grams 문자열이 더해진 리스트가 될 것입니다.

In [9]:
import torch
import torchtext
from torchtext.datasets import text_classification
import os

In [271]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'

## 2)  한국어 뉴스 데이터셋을 전처리해봅시다

`AGNEWS` 데이터셋에 맞게 전처리를 하기 위해 이와 같은 과정이 필요하다
1) 각 단어들을 tokenize함 <br>
2) uni-gram + bi-gram 형태의 리스트로 만들어줌<br>
3) \<sos> 토큰과 \<eos> 토큰을 추가해줌<br> 
4) 토큰을 숫자로 바꿔 줌.<br>
5) tensor로 바꿔 줌.<br>
이를 쉽게 하기 위하여 `torchtext.data`의 `Field`를 사용해보자

*class* <b>torchtext.data.Field</b>(sequential=True, use_vocab=True, init_token=None, eos_token=None, fix_length=None, dtype=torch.int64, preprocessing=None, postprocessing=None, lower=False, tokenize=None, tokenizer_language='en', include_lengths=False, batch_first=False, pad_token='<pad>', unk_token='<unk>', pad_first=False, truncate_first=False, stop_words=None, is_target=False)

어떻게 text processing을 할 것인지에 대해 나타냄. `Vocab`이란 object를 가지고 있어 token들과 token의 숫자표현을 담고 있음. `Field` object는 또한 데이터타입이 어떻게 numericalize되는지에 대한 방법도 담고 있음.

In [16]:
from torchtext.data import Field
from torchtext.data.utils import ngrams_iterator

In [272]:
field = Field(init_token = '<sos>',
              eos_token = '<eos>')

/home/long8v/anaconda3/envs/long36v/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [18]:
corpus = ' '.join([' '.join(s) 
                   for sub in subject.values() 
                   for s in sub])

In [276]:
# ngrams_iterator를 하면 uni-gram + bi-gram으로 나옴
list(ngrams_iterator('안녕 나는 강북 멋쟁이'.split(), 2))

['안녕', '나는', '강북', '멋쟁이', '안녕 나는', '나는 강북', '강북 멋쟁이']

In [19]:
corpus_split = corpus.split()
corpus_ngram = list(ngrams_iterator(corpus_split, 2)) 

<class 'list'>


In [279]:
field.build_vocab([corpus_ngram])
vocab = field.vocab

In [281]:
vocab['트럼프']

33

In [282]:
vocab['트럼프 대통령']

4074

In [283]:
vocab['트럼프 회장']

0

In [24]:
# 원래 예제의 형태처럼 category와 텐서화된 문장을 리스트로 넣어줌
subject_numerical = []
for key, articles in subject.items():
    for article in articles:
        article_ngram = list(ngrams_iterator(article, 2))
        numeric_v = torch.Tensor([vocab[token] for token in article_ngram]).type(torch.int64)
        subject_numerical.append((key, numeric_v))

In [25]:
subject_numerical[0]

(0,
 tensor([ 2432,  1067,  1722,   485, 24260, 28391, 43915,  1573, 35213,    15,
          1436,  1067,   568, 10775,  1692,  8170,   978,   433,  3472,  4150,
          2564,  1722,   279,   216, 24278,  7234, 16343,   896,  3595,     7,
           433,  3472,  1573,   145,    19, 15387,  2930,   313,   976,   279,
           238,  7661,  2564,  1722,   216, 24264,  1142,   391,   184,  5726,
         17400,   672,   145, 15633,  1819,    15,  8593,  2848,    13,  2517,
          2803, 28063,   564, 30855, 15607,   986,  3190,  9610,   718,  2874,
            37, 22222,   216, 24281,   213,  8609, 39143,  1072,  6558,   156,
            45, 15577,   366,   534,  3020,  1069,   218,    15,  3149,   139,
          3508,   534,  3520, 25311,   366,  5676,  3336,   534,  3312,   623,
          1428,  3510,  4199,  3117,  1023,  2774, 26931,    11,   145, 15684,
          3040,  1427,  1861,  3094,   715,  3050,  3884,  3415,  3001, 45938,
         15641,  3310,   834,  4232,  3171,   55

In [26]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [212]:
train, test = train_test_split(subject_numerical)

In [213]:
class VainallaDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]
    def get_vocab(self):
        return field.vocab
    def get_labels(self):
        return list(subject.keys())

In [214]:
train_ds = VainallaDataset(train)
test_ds = VainallaDataset(test)

## 3) 모델을 무사히 돌려봅시다 

### 모델 정의하기

우리의 모델은
`EmbeddingBag`레이어와 선형 레이어로 구성됩니다 (아래 그림 참고).
``nn.EmbeddingBag``는 임베딩들로 구성된 '가방'의 평균을 계산합니다.
이때 텍스트(text)의 각 원소는 그 길이가 다를 수 있습니다. 텍스트의
길이는 오프셋(offset)에 저장되어 있으므로 여기서 ``nn.EmbeddingBag``
에 패딩을 사용할 필요는 없습니다.

덧붙여서, ``nn.EmbeddingBag`` 은 임베딩의 평균을 즉시 계산하기 때문에,
텐서들의 시퀀스를 처리할 때 성능 및 메모리 효율성 측면에서의 장점도
갖고 있습니다.

![](../_static/img/text_sentiment_ngrams_model.png)





In [215]:
import torch.nn as nn
import torch.nn.functional as F

In [217]:
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

### 인스턴스 생성하기
크롤링한 뉴스 데이터셋에는 4 종류의 레이블이 달려 있으며, 따라서 클래스의 개수도 4개 입니다.

어휘집의 크기(Vocab size)는 어휘집(vocab)의 길이와 같습니다 (여기에는
각각의 단어와 ngram이 모두 포함됩니다).

In [218]:
VOCAB_SIZE = len(train_ds.get_vocab()) # 이를 하기 위해 Dataset을 정의할 때 get_vocab이란 메소드를 만들어 줌
EMBED_DIM = 32
NUN_CLASS = len(train_ds.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

In [219]:
len(test_ds.get_vocab())

47293

### 배치 생성을 위한 함수들

텍스트 원소의 길이가 다를 수 있으므로, 데이터 배치와 오프셋을 생성하기
위한 사용자 함수 generate_batch()를 사용하려 합니다. 이 함수는
``torch.utils.data.DataLoader`` 의 ``collate_fn`` 인자로 넘겨줍니다.

``collate_fn`` 의 입력은 그 크기가 batch_size인 텐서들의 리스트이며,
``collate_fn`` 은 이들을 미니배치로 묶는 역할을 합니다. 여러분이
주의해야 할 점은, ``collate_fn`` 를 선언할 때 최상위 레벨에서 정의해야
한다는 점입니다. 그래야 이 함수를 각각의 워커에서 사용할 수 있음이
보장됩니다.

원본 데이터 배치 입력의 텍스트 원소들은 리스트 형태이며, 이들을 하나의
텐서가 되도록 이어 붙인 것이 ``nn.EmbeddingBag`` 의 입력이 됩니다.
오프셋은 <b>텍스트의 경계를 나타내는 텐서</b>이며, 각 원소가 텍스트 텐서의
어느 인덱스에서 시작하는지를 나타냅니다. 레이블은 각 텍스트 원소의
레이블을 담고 있는 텐서입니다.




<b>collate_fn (callable, optional)</b>: <br>merges a list of samples to form a
        mini-batch of Tensor(s).  Used when using batched loading from a
        map-style dataset.

### offsets 예제 

In [294]:
# an Embedding module containing 10 tensors of size 3
embedding_sum = nn.EmbeddingBag(10, 3, mode='sum') 
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([0,1,2,4,5,4,3,2,9])
offsets = torch.LongTensor([0,5,8]) # 0, 1~5, 6~8이 한개의 문서다
embedding_sum(input, offsets) 

tensor([[-0.4510, -2.5885,  3.0341],
        [-0.4592, -0.2884,  1.3650],
        [-0.2143, -1.9077, -1.3087]], grad_fn=<EmbeddingBagBackward>)

In [288]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text] # [0, 첫번째 텍스트의 길이, 두번째 텍스트의 길이 ...]
    # torch.Tensor.cumsum은 dim 차원의 요소들의 누적 합계를 반환합니다.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0) # [0, 첫번째 텍스트가 끝나는 인덱스, ...모델을]
    text = torch.cat(text)
    return text, offsets, label

### 모델을 학습하고 결과를 평가하는 함수 정의하기

PyTorch 사용자라면
`torch.utils.data.DataLoader`를 활용하는 것을 추천합니다. 또한 이를 사용하면 데이터를 쉽게 병렬적으로
읽어올 수 있습니다 (이에 대한 튜토리얼은 `이 문서 <https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html>`
를 참고하시기 바랍니다). 우리는 여기서 ``DataLoader`` 를 이용하여
AG_NEWS 데이터셋을 읽어오고, 이를 모델로 넘겨 학습과 검증을 진행합니다.




In [289]:
data = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)

In [290]:
def train_func(sub_train_):
    # Train the model
    # 모델을 학습합니다
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # 학습율을 조절합니다
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

### 데이터셋을 분할하고 모델 수행하기


원본 AG_NEWS에는 검증용 데이터가 포함되어 있지 않기 때문에, 우리는 학습
데이터를 학습 및 검증 데이터로 분할하려 합니다. 이때 데이터를 분할하는
비율은 0.95(학습)와 0.05(검증) 입니다. 우리는 여기서 PyTorch의
핵심 라이브러리 중 하나인
`torch.utils.data.dataset.random_split`함수를 사용합니다.

`CrossEntropyLoss`기준(criterion)은 각 클래스에 대해 nn.LogSoftmax()와 nn.NLLLoss()를
합쳐 놓은 방식입니다.
`SGD` optimizer는 확률적 경사 하강법를 구현해놓은 것입니다. 처음의 학습율은
4.0으로 두었습니다. 매 에폭을 진행하면서 학습율을 조절할 때는
`StepLR`을 사용합니다.




In [224]:
import time
from torch.utils.data.dataset import random_split

In [252]:
N_EPOCHS = 100
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_ds) * 0.95)
sub_train_, sub_valid_ = random_split(train_ds, [train_len, len(train_ds) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60
    if (epoch+1) % 5 == 0:
        print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
        print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
        print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 5  | time in 0 minutes, 0 seconds
	Loss: 0.0784(train)	|	Acc: 46.6%(train)
	Loss: 0.4779(valid)	|	Acc: 80.0%(valid)
Epoch: 10  | time in 0 minutes, 0 seconds
	Loss: 0.0713(train)	|	Acc: 83.0%(train)
	Loss: 0.5170(valid)	|	Acc: 40.0%(valid)
Epoch: 15  | time in 0 minutes, 0 seconds
	Loss: 0.0666(train)	|	Acc: 80.7%(train)
	Loss: 0.4836(valid)	|	Acc: 40.0%(valid)
Epoch: 20  | time in 0 minutes, 0 seconds
	Loss: 0.0635(train)	|	Acc: 87.5%(train)
	Loss: 0.4774(valid)	|	Acc: 40.0%(valid)
Epoch: 25  | time in 0 minutes, 0 seconds
	Loss: 0.0627(train)	|	Acc: 85.2%(train)
	Loss: 0.4747(valid)	|	Acc: 40.0%(valid)
Epoch: 30  | time in 0 minutes, 0 seconds
	Loss: 0.0603(train)	|	Acc: 86.4%(train)
	Loss: 0.4803(valid)	|	Acc: 40.0%(valid)
Epoch: 35  | time in 0 minutes, 0 seconds
	Loss: 0.0599(train)	|	Acc: 86.4%(train)
	Loss: 0.4795(valid)	|	Acc: 40.0%(valid)
Epoch: 40  | time in 0 minutes, 0 seconds
	Loss: 0.0601(train)	|	Acc: 86.4%(train)
	Loss: 0.4785(valid)	|	Acc: 40.0%(valid)
Epoch: 45

### 평가 데이터로 모델 평가하기

In [253]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_ds)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0764(test)	|	Acc: 58.1%(test)


평가 데이터셋을 통한 결과를 확인합니다...



### 임의의 뉴스로 평가하기


In [254]:
import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

In [255]:
news_label = subject_number

In [264]:
news_label

{0: 'society', 1: 'politics', 2: 'economic', 3: 'foreign'}

In [265]:
def predict(text, model, vocab, ngrams):
    tokenizer = lambda e: e.split()
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() 

In [266]:
def predict_string(string, model, train_ds):
    vocab = train_ds.get_vocab()
    model = model.to("cpu")
    print("This is a %s news" %news_label[predict(string, model, vocab, 2)])

In [267]:
predict_string('이건희', model, train_ds)

This is a politics news


In [268]:
predict_string('환매', model, train_ds)

This is a foreign news


In [269]:
predict_string('이슬람', model, train_ds)

This is a economic news


In [270]:
predict_string('대통령', model, train_ds)

This is a society news
